In [ ]:
import requests
import pandas as pd
import numpy as np
import joblib
import shap
import plotly.graph_objects as go
from datetime import datetime
shap.initjs()

In [ ]:
try:
    customers, sales = joblib.load('customers.joblib'), joblib.load('sales.joblib')
except:
    customers, sales = pd.read_excel('ucy_eko_data.xlsx', sheet_name='smile_customers'), pd.read_excel('ucy_eko_data.xlsx', sheet_name='smile_sales')
    joblib.dump(customers, 'customers.joblib')
    joblib.dump(sales, 'sales.joblib')

In [ ]:
with pd.ExcelWriter("ui-test.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    sales[sales['ciid'].isin(['5665633CF0744F28A8FD9DBA949E2409', '6258CF24B0C543D295A540CA327DE4CC'])].to_excel(writer, sheet_name="smile_sales", index=False)
    customers[customers['ciid'].isin(['5665633CF0744F28A8FD9DBA949E2409', '6258CF24B0C543D295A540CA327DE4CC'])].to_excel(writer, sheet_name="smile_customers", index=False)

In [ ]:
customers[customers['accreccreateddate'].dt.year==2022]

In [ ]:
user = '6258CF24B0C543D295A540CA327DE4CC'
sales_cut = sales[sales['ciid']==user].drop('ciid', axis=1)
sales_cut

In [ ]:
user = '6A0244DF519548A59309CBB153D03624' # few data
# user = '794E3FB06AA94F24B16AD0FB9AFC5D31' # much data
# user = '159A08CEB97647189FF0C2F7AFC87F1C' # outdated
import json

sales_cut = sales[sales['ciid']==user].drop('ciid', axis=1)
# sales_cut['receiptdate'] = sales['receiptdate'].apply(datetime.timestamp)
sales_cut['receiptdate'] = sales['receiptdate'].apply(str)
sales_cut = sales_cut.to_dict(orient='list')
# sales_cut = json.dumps(sales_cut)
customers_cut = customers[customers['ciid']==user].drop('ciid', axis=1).dropna(axis=1)
customers_cut['accreccreateddate'] = customers_cut['accreccreateddate'].apply(str)
customers_cut = customers_cut.to_dict(orient='list')
# customers_cut = json.dumps(customers_cut)

In [ ]:
request = requests.post(
    'http://0.0.0.0:8000/predict',
    json={
        'user': {'ciid': user},
        'sales': sales_cut,
        'customers': customers_cut,
        'request_fields': {'fields': ['prediction', 'confidence', 'shapley_values']}
    }
)

In [ ]:
response = request.json()
shapley_values = np.array(response['shapley_values']['shapley_values'])
X = pd.DataFrame(response['shapley_values']['X'])
y = pd.DataFrame(response['shapley_values']['y'])
ev = response['shapley_values']['shapley_expected_value']

In [ ]:
shapley_values_explainer = shap.Explanation(
    values=shapley_values,
    data=X.values,
    base_values=np.array([ev]*X.shape[0], dtype=np.float32),
    feature_names=X.columns
)

In [ ]:
request = requests.post(
    'http://0.0.0.0:8000/predict_cluster',
    json={
        'user': {'ciid': user},
        'sales': sales_cut,
        'customers': customers_cut
    }
)

In [ ]:
response = request.json()
cluster = response['cluster']
label = response['label']
similarities = response['similarities']
clusters_mapping = response['clusters_mapping']

In [ ]:
response

In [ ]:
customers[customers['accreccreateddate'] < datetime(year=2021, month=6, day=1)]

In [ ]:
import pickle, json
from services.app_api.features.extractor import FeatureExtractor
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist
with open('services/app_api/configs/centroids_table.table.json', 'r') as f:
    centroids_table = json.load(f)
centroids = centroids_table['data']
with open('services/app_api/features/winsorizing_object_for_threshold.pkl', 'rb') as f:
    winsor = pickle.load(f)
sales_cut = sales[sales['ciid']==user]
customers_cut = customers[customers['ciid']==user]
fe = FeatureExtractor(generation_type='continuous', filtering_set='customers', period=60, subperiod=60, perform_split=False)
X, _ = fe.transform(sales_cut, customers_cut)
X.columns = [f'{col[:col.find("_1-60")]}' for col in X.columns]
X = X[['monetary', 'recency', 'average_days_between_visits']]
# outliers winsorization
monetary_threshold = winsor.max()
X.loc[X['monetary'] > monetary_threshold, 'monetary'] = monetary_threshold
# Assure proper scaling (ref: https://stackoverflow.com/questions/46555820/sklearn-standardscaler-returns-all-zeros)
X = X.to_numpy()[0]
X = StandardScaler().fit_transform(X[:, np.newaxis])
# Reshape: 1 col 3 rows => 3 cols 1 row
# to assure proper distance computation
X = X.reshape(1, -1)
euclidean_distances = cdist(X, centroids, 'euclidean')
similarities = np.exp(-euclidean_distances).astype(np.float64)
cluster = similarities[0].tolist().index(similarities.max())

In [ ]:
X

In [ ]:
centroids

In [ ]:
euclidean_distances

In [ ]:
similarities

In [ ]:
similarities/similarities.sum()

In [ ]:
customers

In [ ]:
similarities = similarities/total_score
similarities

In [ ]:
from services.app_api.features.extractor import FeatureExtractor
from plotly import graph_objects as go
import plotly.express as px
import inflect

In [ ]:
fe = FeatureExtractor(generation_type='continuous', filtering_set='customers', period=60, subperiod=30)

In [ ]:
df = fe.filter_sales(sales=sales, customers=customers)
df = fe.extract_subperiods(df)
df['months_enum'] = df['breaks'].apply(lambda x: int(int(x[x.find('-')+len('-'):])/30))

In [ ]:
df['ciid'].unique().shape

In [ ]:
df_chart = df.drop_duplicates(['ciid', 'receiptid'])\
    .groupby(['months_enum'])['receiptid'].count()\
        .to_frame('Total Visits per Month')\
            .reset_index()\
                .rename({'months_enum': 'Month Since First Visit'}, axis=1)
num_convert = inflect.engine()
df_chart['Month Since First Visit'] = df_chart['Month Since First Visit'].apply(lambda x: f'{num_convert.ordinal(x)}')

In [ ]:
LAST_MONTH = 12
cmap = px.colors.sequential.dense
fig = go.Figure(
    go.Funnel(
        y=df_chart.iloc[:LAST_MONTH, :]['Month Since First Visit'],
        x=df_chart.iloc[:LAST_MONTH, :]['Total Visits per Month'],
        textinfo='value+percent initial',
        marker={
            'color': cmap
        }
    )
)
fig.update_layout(
    title='Visits Funnel Throughout User Lifetime (users registered after June 1, 2021)',
    yaxis_title='Month Since First Visit'
)
fig.show()

In [ ]:
import json
fig_json = fig.to_plotly_json()
fig_json['data'][0]['x'] = fig_json['data'][0]['x'].tolist()
fig_json['data'][0]['y'] = fig_json['data'][0]['y'].tolist()

with open('service/app_ui/static/funnel.json', 'w') as f:
    json.dump(fig_json, f)

In [ ]:
def extract_days_between_visits(sales, subperiod = None):
    '''
    Method to extract the average number of days between visits feature
    '''
    def compute_max(s):
        # NaT occur if customer made only one visit
        if type(s) == pd._libs.tslibs.nattype.NaTType:
            return 0
        else:
            # return s[~pd.isnull(s)].days.sum()/s.shape[0]
            return s[~pd.isnull(s)].days.max()

    # Visit - unique `receiptdate` entry
    # Therefore, drop duplicates of this column for each user
    if subperiod:
        tmp = sales.drop_duplicates(['ciid', 'receiptdate'])\
                .sort_values(['ciid', 'receiptdate'])\
                    .groupby(['ciid', 'breaks'])\
                        .agg(days_between_visits = pd.NamedAgg('receiptdate', pd.Series.diff))\
                            .reset_index()
    else:
        tmp = sales.drop_duplicates(['ciid', 'receiptdate'])\
                .sort_values(['ciid', 'receiptdate'])\
                    .groupby('ciid')\
                        .agg(days_between_visits = pd.NamedAgg('receiptdate', pd.Series.diff))\
                            .reset_index()
    tmp['max_days_between_visits'] = tmp['days_between_visits'].apply(compute_max)

    sales = pd.merge(
        sales,
        tmp[['ciid', 'max_days_between_visits']],
        how='left',
        on='ciid'
    )
    return sales

In [ ]:
df = extract_days_between_visits(sales)
# df = extract_days_between_visits(df)

In [ ]:
df['ciid'].unique().shape[0]

In [ ]:
sales

In [ ]:
df_chart = df.pivot_table(values='max_days_between_visits', index='ciid', aggfunc='mean')

In [ ]:
SUBPERIOD = np.floor(df_chart['max_days_between_visits'].max()/7).astype(int)
new_series = pd.cut(df_chart['max_days_between_visits'], bins=SUBPERIOD)
weeks = 1
labels = [f'<= {weeks} Week'] # whatever `period` value is, first one will always be 1-`days`
lower = 0
upper = weeks
for i in range(1, len(new_series.cat.categories)):
    lower += weeks
    upper += weeks
    labels.append(f'{lower} to {upper} Weeks')
new_series = new_series.cat.rename_categories(labels)

In [ ]:
df_chart = pd.merge(
    df_chart,
    pd.DataFrame(new_series).reset_index().rename({'max_days_between_visits': 'max_weeks_between_visits'}, axis=1),
    how='left',
    on='ciid'
)

In [ ]:
def combine_categories(s):
    try:
        val = int(s[:s.find(' ')])
        if val >= 12:
            return 'More than 12 weeks'
        else:
            return s
    except ValueError:
        return s
df_chart['max_weeks_between_visits'] = df_chart['max_weeks_between_visits'].apply(lambda x: combine_categories(x))

In [ ]:
chart_values = df_chart['max_weeks_between_visits'].value_counts()
fig = go.Figure(
    [
        go.Bar(
            x=chart_values.values,
            y=chart_values.index,
            text=chart_values.values,
            textposition='outside',
            orientation='h'
        )
    ]
)
# horizontal line
fig.add_trace(
    go.Scatter(
        x=[int(chart_values[chart_values.index!='More than 12 weeks'].sum()), int(chart_values[chart_values.index!='More than 12 weeks'].sum())],
        y=[labels[0], labels[11]],
        orientation='h',
        mode='lines',
        line={'dash': 'dot'},
    )
)
# text (value)
fig.add_trace(
    go.Scatter(
        x=[int(chart_values[chart_values.index!='More than 12 weeks'].sum())],
        y=[labels[6]],
        text=f"{chart_values[chart_values.index!='More than 12 weeks'].sum():,}",
        orientation='h',
        mode='text',
        textposition='middle right'
    )
)
# left vertical line
fig.add_shape(
    type='line',
    x0=0,
    x1=int(chart_values[chart_values.index!='More than 12 weeks'].sum()),
    y0=labels[0],
    y1=labels[0],
    line={
        'color': 'red',
        'width': 2,
        'dash': 'dot'
    }
)
# right vertical line
fig.add_shape(
    type='line',
    x0=0,
    x1=int(chart_values[chart_values.index!='More than 12 weeks'].sum()),
    y0=labels[11],
    y1=labels[11],
    line={
        'color': 'red',
        'width': 2,
        'dash': 'dot'
    }
)
# remove legend
for trace in fig['data']: 
    trace['showlegend'] = False
fig.data[0]['text'] = ['' if i != chart_values['More than 12 weeks'] else f'{int(i):,}' for i in fig.data[0]['text']]
fig.update_yaxes(
    categoryorder='array',
    categoryarray=new_series.cat.categories[:12].tolist()+['More than 12 weeks']
)
fig.update_layout(
    title='Maximum Breaks between Visits',
    yaxis_title='Number of Users',
    xaxis_title='Break Duration'
)

In [ ]:
fig_json = fig.to_plotly_json()
fig_json['data'][0]['x'] = fig_json['data'][0]['x'].tolist()
fig_json['data'][0]['y'] = fig_json['data'][0]['y'].tolist()

with open('service/app_ui/static/max_breaks_horizontal.json', 'w') as f:
    json.dump(fig_json, f)

In [ ]:
from plotly.subplots import make_subplots

sales['year'] = sales['receiptdate'].dt.year
sales['month'] = sales['receiptdate'].dt.month
sales['month_name'] = sales['receiptdate'].dt.month_name()

col_map = {'month_year': 'Month', 'receiptid': 'Total Visits', 'ciid': 'Total Unique Users'}

monthly_visits = sales.\
    groupby(['year', 'month', 'month_name'], as_index=False).\
    agg({'receiptid': 'count', 'ciid': pd.Series.nunique})
monthly_visits.sort_values(['year', 'month'], ascending=True, inplace=True)
monthly_visits['month_year'] = monthly_visits['month_name'] + ' ' + monthly_visits['year'].astype(str)
monthly_visits.rename(columns=col_map, inplace=True)

period_start, period_end = sales['receiptdate'].min(), sales['receiptdate'].max()

fig = make_subplots(specs=[[{'secondary_y': True}]])

## Unique visits chart
fig.add_trace(
    go.Scatter(
        name='Unique visits',
        x=monthly_visits[col_map['month_year']],
        y=monthly_visits[col_map['receiptid']]
    ),
    secondary_y=False
)

## June 2021 straight line
june_2021 = monthly_visits[col_map['month_year']][monthly_visits[col_map['month_year']]=='June 2021']
fig.add_trace(
    go.Scatter(
        x=['June 2021', 'June 2021'],
        name='',
        mode='lines',
        y=[
            0,
            150000
        ],
        line={'dash': 'dot'}
    )
)
## Unique users chart
fig.add_trace(
    go.Scatter(
        name='Unique users',
        x=monthly_visits[col_map['month_year']],
        y=monthly_visits[col_map['ciid']],
        line={'dash': 'dash'}
    ),
    secondary_y=True
)
## Visits dot
fig.add_trace(
    go.Scatter(
        mode='lines+markers+text',
        x=june_2021,
        y=monthly_visits[col_map['receiptid']][june_2021.index],
        marker={'size': 8, 'color': 'blue'},
        text=f"{monthly_visits[col_map['receiptid']][june_2021.index].values[0]:,} unique visits",
        textposition='top left'
    ),
    secondary_y=False
)
## Users dot
fig.add_trace(
    go.Scatter(
        mode='lines+markers+text',
        x=june_2021,
        y=monthly_visits[col_map['ciid']][june_2021.index],
        marker={'size': 8, 'color': 'green'},
        text=f"{monthly_visits[col_map['ciid']][june_2021.index].values[0]:,} unique users",
        textposition='bottom right'
    ),
    secondary_y=True
)
## Settings
# x axis
fig.update_xaxes(
    tickangle=-45,
    tickvals=[i for i in range(0, monthly_visits.shape[0]) if i % 2 == 1]
)
# vists y axis
fig.update_yaxes(
    title_text='Visits, thousands',
    tickvals=[i for i in range(1000, 201000, 1000) if i % 10000 == 0],
    range=[
        monthly_visits[col_map['receiptid']].min(),
        monthly_visits[col_map['receiptid']].max()
    ],
    secondary_y=False
)
# users y axis
fig.update_yaxes(
    title_text='Users, thousands',
    tickvals=[i for i in range(1000, 41000, 1000) if i % 5000 == 0],
    range=[
        monthly_visits[col_map['ciid']].min(),
        monthly_visits[col_map['ciid']].max()
    ],
    secondary_y=True
)
# layout
fig.update_layout(
    title=f'Monthly Visits from {period_start.month_name()} {period_start.day}, {period_start.year} to {period_end.month_name()} {period_end.day}, {period_end.year}',
    showlegend=True
)
# Remove extras from legend
for trace in fig['data']:
    if not trace['name']:
        trace['showlegend'] = False
fig.show()

In [ ]:
fig_json = fig.to_plotly_json()
fig_json['data'][0]['x'] = fig_json['data'][0]['x'].tolist()
fig_json['data'][0]['y'] = fig_json['data'][0]['y'].tolist()

fig_json['data'][2]['x'] = fig_json['data'][2]['x'].tolist()
fig_json['data'][2]['y'] = fig_json['data'][2]['y'].tolist()

fig_json['data'][3]['x'] = fig_json['data'][3]['x'].tolist()
fig_json['data'][3]['y'] = fig_json['data'][3]['y'].tolist()

fig_json['data'][4]['x'] = fig_json['data'][4]['x'].tolist()
fig_json['data'][4]['y'] = fig_json['data'][4]['y'].tolist()

with open('service/app_ui/static/monthly_visits.json', 'w') as f:
    json.dump(fig_json, f)

In [ ]:
fe = FeatureExtractor(target_month=3, n_purchases=2, perform_split=False, generation_type='continuous', filtering_set='customers', period=60, subperiod=60)
df, y_2 = fe.transform(sales, customers)
df['target_2_visits'] = y_2
fe = FeatureExtractor(target_month=3, n_purchases=1, perform_split=False, generation_type='continuous', filtering_set='customers', period=60, subperiod=60)
_, y_1 = fe.transform(sales, customers)
df['target_1_visit'] = y_1

In [ ]:
df.columns = [f'{col[:col.find("_1-60")]}' if '_1-60' in col else col for col in df.columns]
df.head()

In [ ]:
df_agg = pd.concat(
    [
        df.groupby('target_1_visit')[['average_days_between_visits', 'monetary', 'recency']].median(),
        df.groupby('target_2_visits')[['average_days_between_visits', 'monetary', 'recency']].median()
    ],
    axis=0
)
df_agg.index = [
    'No visits at month 3',
    'At least 1 visit at month 3',
    'No or 1 visit at month 3',
    'At least 2 visits at month 3',
]
df_agg

In [ ]:
fig = go.Figure(
    go.Heatmap(
        x=df_agg.columns,
        y=df_agg.index,
        z=df_agg.values,
        text=df_agg.values.astype(str),
        texttemplate='%{text:.2f}',
    )
)
fig.show()

In [ ]:
customers.isna().sum()*100 / customers.shape[0]

In [ ]:
sales[
    sales['ciid'].isin(customers[(~customers['cigender'].isna()) & (~customers['ciyearofbirth'].isna())]['ciid'])
]

In [ ]:
(677380 / sales.shape[0])*100

In [ ]:
import pickle
from sklearn.preprocessing import StandardScaler

def extract_clustering_feature(df_customer_level: pd.DataFrame):
    '''
    Method to extract clusters (i.e. customer segments) based on RFM variables
    TODO: add extended docstring
    '''
    # Load clustering model
    with open('service/app_api/features/clustering_model.pkl', 'rb') as f:
        model = pickle.load(f)
    # Load `scipy.stats.mstats.winsorize` output object to define threshold for the `monetary` variable
    with open('service/app_api/features/winsorizing_object_for_threshold.pkl', 'rb') as f:
        winsor = pickle.load(f)
    X_clust = df_customer_level[['monetary', 'recency', 'average_days_between_visits']]
    monetary_threshold = winsor.max()
    # Perform winsorization
    X_clust.loc[X_clust['monetary'] > monetary_threshold, 'monetary'] = monetary_threshold
    scaler = StandardScaler()
    labels = pd.Categorical(
        model.predict(
            scaler.fit_transform(X_clust)
        )
    )
    df_customer_level['segments'] = labels
    df_customer_level['segments'] = df_customer_level['segments'].cat.rename_categories({2: 'frequent_drivers', 1: 'passerbys', 0: 'regular_drivers'})
    return df_customer_level

In [ ]:
from services.app_api.features.extractor import FeatureExtractor
fe = FeatureExtractor(target_month=3, n_purchases=2, perform_split=False, generation_type='continuous', filtering_set='customers', period=60, subperiod=60)
X, y = fe.transform(sales=sales, customers=customers)

In [ ]:
X

In [ ]:
l = [0.331075,
-0.041019,
-0.013123,
0.014812,
-0.019510,
-0.020479,
0.033687,
0.034522,
-0.037530,
0.044057,
0.090234]
sum(l)